In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [155]:
url = 'https://en.wikipedia.org/wiki/World%27s_Best_(film)'

response = requests.get(url)

In [156]:
soup = BeautifulSoup(response.content, 'html.parser')

In [157]:
infobox = soup.find('table', {'class': 'infobox'})

In [158]:
data = {}

for row in infobox.find_all('tr'):
    header = row.find('th')
    value = row.find('td')
    if header and value:
        data[header.get_text(' ').strip()] = value.get_text().strip()


In [159]:
data

{'Directed by': 'Roshan Sethi',
 'Written by': 'Utkarsh AmbudkarJamie King',
 'Produced by': 'Thomas KailUtkarsh AmbudkarKate Sullivan',
 'Starring': 'Utkarsh Ambudkar\nManny Magnus\nPunam Patel',
 'Cinematography': 'Sing Howe Yam',
 'Edited by': 'Jonah Moran',
 'Music by': 'Jongnic Bontemps\nRaashi Kulkarni',
 'Production company': 'Walt Disney Pictures',
 'Distributed by': 'Disney+',
 'Release date': 'June\xa023,\xa02023\xa0(2023-06-23)',
 'Running time': '101 minutes',
 'Country': 'United States',
 'Language': 'English'}

In [54]:
for key in data.keys():
    data[key] = data[key].replace('\n',', ')
    data[key] = data[key].replace('\xa0',' ')
    data[key] = data[key].replace('[1]','')
    data[key] = data[key].replace('[a]','')
    
    
    
data
    

{'Directed by': 'Lee Unkrich',
 'Screenplay by': 'Michael Arndt',
 'Story by': 'John Lasseter, Andrew Stanton, Lee Unkrich',
 'Produced by': 'Darla K. Anderson',
 'Starring': 'Tom Hanks, Tim Allen, Joan Cusack, Don Rickles, Wallace Shawn, John Ratzenberger, Estelle Harris, Ned Beatty, Michael Keaton, Jodi Benson, John Morris',
 'Cinematography': 'Jeremy Lasky, Kim White',
 'Edited by': 'Ken Schretzmann',
 'Music by': 'Randy Newman',
 'Production company': 'Pixar Animation Studios',
 'Distributed by': 'Walt Disney StudiosMotion Pictures',
 'Release dates': 'June 12, 2010 (2010-06-12) (Taormina Film Fest), June 18, 2010 (2010-06-18) (United States)',
 'Running time': '103 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million',
 'Box office': '$1.067 billion'}

Getting all the movie links from wikipedia

In [2]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')


In [75]:
sp = BeautifulSoup(r.content, 'html.parser')


In [76]:
movie_links = []


#Getting all the tables from 1930s-2020s
tables = sp.find_all('table')[:9]

for table in tables:
    
    
    
    # Getting all the rows from a table except the first row
    # because it is a header row
    rows = table.find_all('tr')[1:]
    
    for row in rows:
        
        # Finding the first instance of 'i' in a row
        title_cell = row.find('i')
        
        # Getting the 'href' attribute if the title_cell is not empty
        if title_cell.find('a') != None:
            
            link = ((title_cell.find('a')).attrs)['href']
            movie_links.append('https://en.wikipedia.org' + link + ' ')
        
        
        
        
        
movie_links[-10:]

        

['https://en.wikipedia.org/wiki/World%27s_Best_(film) ',
 'https://en.wikipedia.org/wiki/Indiana_Jones_and_the_Dial_of_Destiny ',
 'https://en.wikipedia.org/wiki/Haunted_Mansion_(2023_film) ',
 'https://en.wikipedia.org/wiki/Dashing_Through_the_Snow ',
 'https://en.wikipedia.org/wiki/Wish_(film) ',
 'https://en.wikipedia.org/wiki/Diary_of_a_Wimpy_Kid_Christmas:_Cabin_Fever ',
 'https://en.wikipedia.org/wiki/Tiger_(2024_film) ',
 'https://en.wikipedia.org/wiki/The_Beach_Boys_(film) ',
 'https://en.wikipedia.org/wiki/Young_Woman_and_the_Sea ',
 'https://en.wikipedia.org/wiki/Inside_Out_2 ']

Checking if all the URLs are working

In [160]:
def check_links(urls):
    valid_links = []
    invalid_links = []

    for url in urls:
        try:
            response = requests.head(url, allow_redirects=True)
            if response.status_code == 200:
                valid_links.append(url)
            else:
                invalid_links.append((url, response.status_code))
        except requests.exceptions.RequestException as e:
            invalid_links.append((url, str(e)))

    return valid_links, invalid_links

In [161]:
#check_links(movie_links)

(['https://en.wikipedia.org/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film) ',
  'https://en.wikipedia.org/wiki/Pinocchio_(1940_film) ',
  'https://en.wikipedia.org/wiki/Fantasia_(1940_film) ',
  'https://en.wikipedia.org/wiki/The_Reluctant_Dragon_(1941_film) ',
  'https://en.wikipedia.org/wiki/Dumbo ',
  'https://en.wikipedia.org/wiki/Bambi ',
  'https://en.wikipedia.org/wiki/Saludos_Amigos ',
  'https://en.wikipedia.org/wiki/Victory_Through_Air_Power_(film) ',
  'https://en.wikipedia.org/wiki/The_Three_Caballeros ',
  'https://en.wikipedia.org/wiki/Make_Mine_Music ',
  'https://en.wikipedia.org/wiki/Song_of_the_South ',
  'https://en.wikipedia.org/wiki/Fun_and_Fancy_Free ',
  'https://en.wikipedia.org/wiki/Melody_Time ',
  'https://en.wikipedia.org/wiki/So_Dear_to_My_Heart ',
  'https://en.wikipedia.org/wiki/The_Adventures_of_Ichabod_and_Mr._Toad ',
  'https://en.wikipedia.org/wiki/Cinderella_(1950_film) ',
  'https://en.wikipedia.org/wiki/Treasure_Island_(1950_film) ',
  'https://e

In [77]:
len(movie_links)

530

### Task 2: Getting infoboxes from all the movie links from wikipedia and store them to movie_infobox_list

In [3]:
import re

In [79]:

def clean_data(data):
    
    
    
    data = data.replace('\n',', ')
    data = data.replace('\xa0',' ')
    data = re.sub(r"\[[^\]]+\]","", data) #Clears everything with & within []
    data = re.sub(r'\(.*?\)',"", data) #Clears everything with & within ()
    
    #Checks if the strings contains an alphanumeric character
    lines = data.split(', ')
    filtered_lines = [line for line in lines if  any(char.isalnum() for char in line )]

    cleaned_data = ', '.join(filtered_lines)
        
    
    if len(cleaned_data) != 0:
        return cleaned_data
    else:
        return None


def get_infobox(url):
    

    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')


    infobox = soup.find('table', {'class': 'infobox'})

    data = {}

    fetch_title = False
    
    for row in infobox.find_all('tr'):
        header = row.find('th')
        value = row.find('td')
        
        if not fetch_title: 
            data['title'] = row.find('th', class_='infobox-above summary').get_text().strip()
            fetch_title = True
        
        if header and value:
            
            # This is for those long joint names of strings
            if value.find('br'):
                data[header.get_text(' ').strip()] =[value.get_text(separator='\n', strip=True)][0].split('\n')
                
            else:
                data[header.get_text(' ').strip()] = value.get_text().strip()
                
                


    for key, value in data.items():
        
        if isinstance(value, list):
            
            data[key] = [clean_data(item) for item in value if clean_data(item) != None]
            
  
        else: 
            
            
            data[key] = clean_data(data[key])
            
            

        
    return data
        

In [80]:
links = ['https://en.wikipedia.org/wiki/The_Reluctant_Dragon_(1941_film)', 'https://en.wikipedia.org/wiki/The_Bears_and_I', 'https://en.wikipedia.org/wiki/Herbie_Rides_Again'] 
test_movie = []

for link in links:

    test_movie.append(get_infobox(link))
    
test_movie



[{'title': 'The Reluctant Dragon',
  'Directed by': ['Alfred Werker',
   'Hamilton Luske',
   'Jack Cutting',
   'Ub Iwerks',
   'Jack Kinney'],
  'Written by': ['Live-action:',
   'Ted Sears',
   'Al Perkins',
   'Larry Clemmons',
   'Bill Cottrell',
   'Harry Clork',
   'Robert Benchley',
   'The Reluctant Dragon',
   'segment:',
   'Kenneth Grahame',
   'Erdman Penner',
   'T. Hee',
   'Baby Weems',
   'segment:',
   'Joe Grant',
   'Dick Huemer',
   'John Miller'],
  'Produced by': 'Walt Disney',
  'Starring': ['Robert Benchley',
   'Frances Gifford',
   'Buddy Pepper',
   'Nana Bryant'],
  'Cinematography': 'Bert Glennon',
  'Edited by': 'Paul Weatherwax',
  'Music by': ['Frank Churchill', 'Larry Morey'],
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release date': 'June 27, 1941 ',
  'Running time': '74 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$600,000',
  'Box office': '$960,000  '},
 {'title'

In [81]:
movie_infoboxes = []

for index, movie in enumerate(movie_links):
    
    if index % 10 == 0:
        print(index)
            
    movie_infoboxes.append(get_infobox(movie))




0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520


Saving movie_infoboxes into a JSON file

In [84]:
import json

In [85]:
with open('output/Prev versions/movie_infoboxes_clean_v2.json', 'w') as json_file:
    json.dump(movie_infoboxes,json_file, indent=4)

In [3]:
with open('output/Prev versions/movie_infoboxes_clean_v2.json', 'r') as file:
    # Read the contents of the file
    data = file.read()

In [293]:
movie_infoboxes_list = json.loads(data)

In [294]:
movie_infoboxes_list[:5]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': 'December 21, 1937  , February 4, 1938  ',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million'},
 {'title': 'Pinocchio',
  'Directed by': 'Supervising Directors, Ben Sharpsteen, Hamilton Luske, Sequence Directors, Bill Roberts, Norman Ferguson, Jack Kinney, Wilfred Jackson, 

### Task 3: Clean data

##### Subtasks

- ~~Clean up references [2][3]~~
- ~~Convert running time into int~~
- ~~Convert dates into datetime~~
- ~~Split up the long strings~~
- ~~Convert budget and box office to float~~  

In [295]:
movie_infoboxes_list[:5]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': 'December 21, 1937  , February 4, 1938  ',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million'},
 {'title': 'Pinocchio',
  'Directed by': 'Supervising Directors, Ben Sharpsteen, Hamilton Luske, Sequence Directors, Bill Roberts, Norman Ferguson, Jack Kinney, Wilfred Jackson, 

 Converting 'Running time' into int


In [296]:
# To convert Running time into int 

def running_time_to_int(dict):

    key = 'Running time'

    if key in dict.keys():
        if isinstance(dict[key], list):
            filterred_str= re.sub(r'[^0-9]', '', dict[key][0][:3])
        
        elif isinstance(dict[key], str):
            filterred_str = re.sub(r'[^0-9]', '', dict[key][:3])

            
        dict[key+'(int)'] = int(filterred_str)



In [298]:
for infobox in movie_infoboxes_list:
    running_time_to_int(infobox) 

In [299]:
movie_infoboxes_list[:5]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': 'December 21, 1937  , February 4, 1938  ',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million',
  'Running time(int)': 83},
 {'title': 'Pinocchio',
  'Directed by': 'Supervising Directors, Ben Sharpsteen, Hamilton Luske, Sequence Directors, Bill Roberts, Norman Ferguson, Jac

Converting 'Box office' & 'Budget' into float

Types of values to consider:

1- '$960,000' <br>
2- '$1.066 billion' <br>
3- 'est. ₹79.43 crore ' <br>
4- "['Original release', '$2.6 million ', '1969 re-release', '$2.3 million  ']" <br>
5- '$76.4–$83.3 million ', <br>
6- '₹1,968−2,500 crore ' <br>
7- "['$38.2 million ', '$30.8 million (worldwide', 'rentals']" <br>
8- '$3.5 to 4 million' <br>


In [254]:
print([movie.get('Budget', 'N/A') for movie in movie_infoboxes_list])

['$1.5 million', '$2.6 million', '$2.28 million', '$600,000', '$950,000', '$858,000', 'N/A', '$788,000', 'N/A', '$1.35 million', '$2.125 million', 'N/A', '$1.5 million', '$1.5 million', 'N/A', '$2.2 million', '$1.8 million', '$3 million', 'N/A', '$4 million', '$2 million', '$300,000', '$1.8 million', 'N/A', '$5 million', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$700,000', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$6 million', 'under $1 million or $1,250,000', 'N/A', '$2 million', 'N/A', 'N/A', '$2.5 million', 'N/A', 'N/A', 'N/A', '$4 million', '$3.6–4 million', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', '$4.4–6 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A'

In [300]:
def values_convert(val):
    # exp = r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(\w+)'
    exp = r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:-–)?\s*'
    unit_exp = r'\b(?:thousand|million|billion|crore)\b'
    
    val_match = re.search(exp, val)
    unit_match = re.search(unit_exp, val)
    
    units = {
        'thousand' : 1000,
        'million' : 1000000,
        'billion' : 1000000000,
        'crore' : 120000
    }
    
    if val_match:
        
        number = float(val_match.group(1).replace(',',''))
        
        
    if unit_match:
        unit = unit_match.group().lower()
        for key, value in units.items():
            if key == unit:
                val = round(number * value, 2)
    else:
        val = number
             
    
    
    return val

In [301]:

for movie in movie_infoboxes_list:
    
    for key in ['Box office', 'Budget']:      
         
        if movie.get(key, 'N/A') !='N/A':
            
            if isinstance(movie[key], list):
            
                movie[key] = ' '.join(movie[key])
            
            movie[key+'(float)'] = values_convert(movie[key])

        
  

In [302]:
movie_infoboxes[:5]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million',
  'Running time(int)': 83,
  'Box office(float)': 418000000.0,
  'Budget(float)': 1500000.0},
 {'title': 'Pinocchio',
  'Directed by': 'Supervising Directors, Ben Sharpsteen, Hamilton Luske, Sequence Directors, Bill Roberts, Norman Ferguson,

Converting 'Release Date' into Datetime datatype 

In [84]:
from datetime import datetime
import copy

Converting lists to strings

In [342]:


for movie in movie_infoboxes_list:

    if movie.get('Release dates', 0) != 0:
                    
        if isinstance(movie['Release dates'], str):
            
            movie['Release dates'] = movie['Release dates'].split(' , ')[0].strip()
            
        elif isinstance(movie['Release dates'], list):
            
            movie['Release dates'] = " ".join(movie['Release dates']).strip()
            
            
            
            

In [353]:
def to_datetime(dict):

    keys = ['Release dates', 'Release date']

    date_formats = ['%B %d, %Y', '%d %B %Y']

    def convert_str(str):
    
        try:
            dict['Release date (Datetime)'] = datetime.strptime(str, date_formats[0]) 
        
        except ValueError:
            try:
                dict['Release date (Datetime)'] = datetime.strptime(str, date_formats[1]) 
                
            except ValueError:
                print("Error")
        
    if dict.get(keys[0], 'N/A') != 'N/A':
        convert_str(dict[keys[0]])
        
    elif dict.get(keys[1], 'N/A') != 'N/A':
        convert_str(dict[keys[1]].strip())


In [354]:
for infobox in movie_infoboxes_list:
    to_datetime(infobox)

Error
Error
Error
Error
Error


In [359]:
movie_infoboxes_list[:5]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': 'December 21, 1937',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million',
  'Running time(int)': 83,
  'Box office(float)': 418000000.0,
  'Budget(float)': 1500000.0,
  'Release date (Datetime)': datetime.datetime(1937, 12, 21, 0, 0)},
 {'title': 'Pinocchio',
  'Directed by'

In [103]:
[type(movie.get('Release date (Datetime)', 'N/A')) for movie in movie_infoboxes_list]

[datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 str,
 str,
 datetime.datetime,
 datetime.datetime,
 datetime.datetime,
 datetim

Saving data 

In [51]:
import pickle

def save_data(name, data):
    
    with open(name, 'wb') as f:
        pickle.dump(data, f)


In [52]:
import pickle


def load_data(name):
    with open(name, 'rb') as f:
        return pickle.load(f)
    
    
    

In [362]:
save_data('output/Prev versions/movie_infoboxes_clean_v3.pickle', movie_infoboxes_list)

In [54]:
movie_infoboxes_list = load_data('output/Prev versions/movie_infoboxes_clean_v3.pickle')

### Task 4: Attach IMBD, meta and Rotten Tomatoes scores

In [2]:
import requests
import os
import urllib

In [48]:


def get_omdb_info(title):
    
    base_url = 'http://www.omdbapi.com/?'
    paramenters = {'apikey': os.environ['omdb_api_key'], 't': title}
    params_encoded = urllib.parse.urlencode(paramenters)
    full_url = base_url + params_encoded
    
    return requests.get(full_url).json()


get_omdb_info('the beach boys')
    

{'Title': 'The Beach Boys',
 'Year': '2024',
 'Rated': 'PG-13',
 'Released': '24 May 2024',
 'Runtime': '113 min',
 'Genre': 'Documentary, Biography, Music',
 'Director': 'Frank Marshall, Thom Zimny',
 'Writer': 'Mark Monroe',
 'Actors': 'The Beach Boys, Mike Love, Brian Wilson',
 'Plot': 'A celebration of the legendary band that revolutionized pop music and created the harmonious sound that personified the California Dream.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 1 Primetime Emmy. 1 nomination total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDRiYmQ3OGQtNDQ4YS00ZmRiLThiNWYtNTBkNWM3ZjYwZTE5XkEyXkFqcGdeQXVyMTM1NjM2ODg1._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.1/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '89%'}],
 'Metascore': 'N/A',
 'imdbRating': '7.1',
 'imdbVotes': '1,450',
 'imdbID': 'tt31922530',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'N/A',
 'Website': 'N/A',

In [75]:
import re
def get_ratings(title):
    
    info = get_omdb_info(title)
        
    ratings = {
        'imdb_rating': info.get('imdbRating', 'N/A'),
        'metascore': info.get('Metascore', 'N/A'),
        'rotten_tomatoes': info.get('Ratings', 'N/A')
    }
    
    for key, value in ratings.items():
        if value != 'N/A':
            
            if isinstance(value, list):
                for val in value:
                    if val['Source'] == 'Rotten Tomatoes':
                        
                        if val.get('Value', 'N/A') != 'N/A':
                            
                            match = re.match(r'\d+', val['Value'])
                            ratings[key] = float(match.group())
                            break
                    else:
                        ratings[key] = 'N/A'
            else:
                ratings[key] = float(value)
    
    return ratings

get_ratings('the beach boys')

{'imdb_rating': 7.1, 'metascore': 'N/A', 'rotten_tomatoes': 89.0}

In [78]:
for index, info in enumerate(movie_infoboxes_list):
    
    if index%10 == 0:
        print(index)
    
    rating_dct = get_ratings(info['title'])
    
    info['IMDb rating'] = rating_dct['imdb_rating']
    info['Meta Score'] = rating_dct['metascore']
    info['Rotten Tomatoes (%)'] = rating_dct['rotten_tomatoes']
    


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520


In [79]:
movie_infoboxes_list[:10]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': 'December 21, 1937',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million',
  'Running time(int)': 83,
  'Box office(float)': 418000000.0,
  'Budget(float)': 1500000.0,
  'Release date (Datetime)': datetime.datetime(1937, 12, 21, 0, 0),
  'IMDb rating': 7.6,
  'Meta Score': 96

### Task 5: Saving as JSON and CSV  

In [81]:
import copy

movie_infoboxes_list_copy = copy.deepcopy(movie_infoboxes_list) 

In [82]:
movie_infoboxes_list_copy[20]

{'title': 'The Sword and the Rose',
 'Directed by': 'Ken Annakin',
 'Screenplay by': 'Lawrence Edward Watkin',
 'Based on': ['When Knighthood Was in Flower',
  'by',
  'Charles Major',
  'James B. Fagan'],
 'Produced by': ['Perce Pearce', 'Walt Disney'],
 'Starring': ['Glynis Johns',
  'James Robertson Justice',
  'Richard Todd',
  'Michael Gough',
  'Jane Barrett',
  'Peter Copley',
  'Ernest Jay',
  'Jean Mercure',
  'D. A. Clarke-Smith',
  'Gérard Oury',
  'Fernand Fabre',
  'Gaston Richer',
  'Rosalie Crutchley',
  'Bryan Coleman'],
 'Cinematography': 'Geoffrey Unsworth',
 'Edited by': 'Gerald Thomas',
 'Music by': 'Clifton Parker',
 'Production company': 'RKO-Walt Disney British Productions Limited',
 'Distributed by': 'RKO Radio Pictures Ltd.',
 'Release date': 'July 23, 1953  ',
 'Running time': '92 minutes',
 'Countries': ['United Kingdom', 'United States'],
 'Language': 'English',
 'Budget': '$2 million',
 'Box office': '$1 million ',
 'Running time(int)': 92,
 'Box office(flo

In [102]:
from datetime import datetime

In [105]:

for info in movie_infoboxes_list_copy:
    
    if isinstance(info.get('Release date (Datetime)', 'N/A'), datetime):
        info['Release date (Datetime)'] = info['Release date (Datetime)'].strftime('%B %d, %y')


In [98]:
movie_infoboxes_list_copy[:10]

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Supervising Directors',
   'David Hand',
   'Sequence Directors',
   'Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': 'Ted Sears, Richard Creedon, Otto Englander, Dick Rickard, Earl Hurd, Merrill De Maris, Dorothy Ann Blank, Webb Smith',
  'Based on': ['Snow White', 'by the', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Music by': 'Frank Churchill, Leigh Harline, Paul Smith',
  'Production company': 'Walt Disney Productions',
  'Distributed by': 'RKO Radio Pictures',
  'Release dates': 'December 21, 1937',
  'Running time': '83 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$1.5 million',
  'Box office': '$418 million',
  'Running time(int)': 83,
  'Box office(float)': 418000000.0,
  'Budget(float)': 1500000.0,
  'Release date (Datetime)': 'December 21, 37',
  'IMDb rating': 7.6,
  'Meta Score': 96.0,
  'Rotten Tomato

In [106]:
import json
with open('output/movie_infoboxes_clean_final.json', 'w') as f:
    json.dump(movie_infoboxes_list_copy,f, indent=4)

In [107]:
import pandas as pd

df = pd.DataFrame(movie_infoboxes_list)

,title,Directed by,Story by,Based on,Produced by,Music by,Production company,Distributed by,Release dates,Running time,...,Producers,Editor,Kanji,Transcriptions,Revised Hepburn,Traditional Chinese,Simplified Chinese,Literal meaning,Original title,Layouts by
0,Snow White and the Seven Dwarfs,"[Supervising Directors, David Hand, Sequence D...","Ted Sears, Richard Creedon, Otto Englander, Di...","[Snow White, by the, Brothers Grimm]",Walt Disney,"Frank Churchill, Leigh Harline, Paul Smith",Walt Disney Productions,RKO Radio Pictures,"December 21, 1937",83 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pinocchio,"Supervising Directors, Ben Sharpsteen, Hamilto...","Ted Sears, Otto Englander, Webb Smith, William...","[The Adventures of Pinocchio, by, Carlo Collodi]",Walt Disney,"Leigh Harline, Paul J. Smith",Walt Disney Productions,RKO Radio Pictures,"February 7, 1940",88 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fantasia,"Samuel Armstrong, James Algar, Bill Roberts, P...","Joe Grant, Dick Huemer",NaN,"Walt Disney, Ben Sharpsteen",See plot,Walt Disney Productions,RKO Radio Pictures,NaN,126 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Reluctant Dragon,"[Alfred Werker, Hamilton Luske, Jack Cutting, ...",NaN,NaN,Walt Disney,"[Frank Churchill, Larry Morey]",Walt Disney Productions,RKO Radio Pictures,NaN,74 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dumbo,"Supervising director , Ben Sharpsteen, Sequenc...","Joe Grant, Dick Huemer","[Dumbo, the Flying Elephant, by, Helen Aberson...",Walt Disney,"[Frank Churchill, Oliver Wallace]",Walt Disney Productions,RKO Radio Pictures,"October 23, 1941",64 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df.to_csv('output/movie_infoboxes_clean_final.csv')